In [15]:
import geopandas as gpd
import pandas as pd
import urllib.request

import matplotlib as pl
%matplotlib inline

In [3]:
fname = "./data/nyc_zip_code_tabulation_areas_polygons.geojson"

df = gpd.read_file(fname)

print(type(df))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [5]:
df.head()

,cartodb_id,objectid,postalcode,po_name,state,borough,st_fips,cty_fips,bldgpostalcode,shape_leng,shape_area,id,geometry
0,173,173,11693,Far Rockaway,NY,Brooklyn,36,047,0,9479.917277,3.497516e+06,http://nyc.pediacities.com/Resource/PostalCode...,"MULTIPOLYGON (((-73.84076 40.62536, -73.84306 ..."
1,48,48,10463,Bronx,NY,Manhattan,36,061,0,7791.517127,3.119702e+06,http://nyc.pediacities.com/Resource/PostalCode...,"MULTIPOLYGON (((-73.91544 40.87559, -73.91509 ..."
2,51,51,10033,New York,NY,Manhattan,36,061,0,29415.607123,1.615605e+07,http://nyc.pediacities.com/Resource/PostalCode...,"MULTIPOLYGON (((-73.93213 40.86945, -73.93128 ..."
3,120,120,11367,Flushing,NY,Queens,36,081,0,33506.126605,7.281015e+07,http://nyc.pediacities.com/Resource/PostalCode...,"MULTIPOLYGON (((-73.83761 40.74324, -73.83516 ..."
4,64,64,10472,Bronx,NY,Bronx,36,005,0,27006.042411,3.096325e+07,http://nyc.pediacities.com/Resource/PostalCode...,"MULTIPOLYGON (((-73.88011 40.83723, -73.87747 ..."


In [10]:
url = 'https://data.cityofnewyork.us/api/views/nu7n-tubp/rows.csv?accessType=DOWNLOAD'
urllib.request.urlretrieve(url, './data/nycDogLicenses.csv')

('./data/nycDogLicenses.csv', <http.client.HTTPMessage at 0x7fe50ee16290>)

In [13]:
dogs = pd.read_csv('./data/nycDogLicenses.csv')
dogs.head()

,RowNumber,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,LicenseIssuedDate,LicenseExpiredDate,Extract Year
0,1,PAIGE,F,2014,American Pit Bull Mix / Pit Bull Mix,NaN,10035,09/12/2014,09/12/2017,2016
1,2,YOGI,M,2010,Boxer,NaN,10465,09/12/2014,10/02/2017,2016
2,3,ALI,M,2014,Basenji,NaN,10013,09/12/2014,09/12/2019,2016
3,4,QUEEN,F,2013,Akita Crossbreed,NaN,10013,09/12/2014,09/12/2017,2016
4,5,LOLA,F,2009,Maltese,NaN,10028,09/12/2014,10/09/2017,2016


In [25]:
dogs.AnimalBirthMonth.value_counts().head()

2016    36151
2015    35316
2014    31119
2013    27275
2012    25098
Name: AnimalBirthMonth, dtype: int64

In [26]:
dogs.AnimalBirthMonth.max()

2018

In [35]:
dogs.ZipCode.unique()

array([10035, 10465, 10013, 10028, 10025, 11215, 11201, 10022, 10003,
       11220, 10040, 10463, 11238, 10002, 10308, 10011, 11209, 11208,
       11218, 11232, 10312, 10023, 11231, 10461, 11224, 10471, 11236,
       11229, 11361, 10021, 10309, 10024, 10038, 10467, 33185, 10012,
       10473, 11104, 10034, 11379, 10029, 10451, 11374, 11106, 10016,
       11435, 10065, 10280, 10031, 10128, 10475, 11230, 10460, 11101,
       11211, 11375, 11228, 10036, 10462, 10458, 10019, 10454, 11373,
       11217, 11237, 11214, 11385, 11102, 10027, 10301, 11222, 11364,
       11234, 11205, 11377, 10314, 10306, 10014, 11206, 10026, 10032,
       11365, 11694, 11221, 11372, 10469, 11358, 11414, 10009, 10033,
       11429, 11378, 11105, 10456, 11249, 11426, 11225, 10282, 10459,
       10075, 11416, 11216, 11355, 10304, 11235, 10001, 10007, 10039,
       11415, 10017, 10452, 11204, 11434, 11418, 10470, 10037, 11233,
       11368, 11219, 11354, 11223, 11207, 11239, 10303, 11226, 11691,
       11423, 11419,

In [45]:
dogsByZip = dogs[dogs.AnimalBirthMonth>=2012].groupby('ZipCode', as_index=False).count()[['ZipCode', 'RowNumber']]

In [48]:
dogsByZip.rename(columns={"ZipCode":"postalcode", "RowNumber":"dogcount"}, inplace=True)

In [55]:
dogsByZip['postalcode'] = dogsByZip['postalcode'].apply(lambda x: '{0:0>5}'.format(x))

In [59]:
dogsByZip = df.merge(dogsByZip, on='postalcode')

In [61]:
print(type(dogsByZip))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [66]:
dogsByZip.drop(['id', 'bldgpostalcode', 'cty_fips', 'st_fips', 'cartodb_id', 'objectid'], axis=1, inplace=True)

In [ ]:
dogsByZip.